In [1]:
from brainbox.task import trials
from one.api import ONE
import pandas as pd
import numpy as np
import GEMA

one = ONE(base_url='https://openalyx.internationalbrainlab.org', password='international', silent=True)
eid = '58b1e920-cfc8-467e-b28b-7654a55d0977'

In [10]:
def get_raster_and_psth(eid):
    #Load trials into df
    dat = one.load_dataset(eid, 'alf/_ibl_trials.table.pqt')
    df=pd.DataFrame(dat)
    #Choose one type of event
    trials_=df.loc[(df['choice'] == 1) & (df['contrastLeft'] == 1.0)].index
    #df=df.loc[(df['choice'] == 1) & (df['contrastLeft'] == 1.0)]
    events=df["stimOn_times"]
    #Get spike_times
    spike_times=one.load_dataset(eid, 'alf/probe00/pykilosort/spikes.times.npy')
    #Get clusters
    spike_clusters=one.load_dataset(eid, 'alf/probe00/pykilosort/spikes.clusters.npy')
    print(len(spike_clusters))
    print(spike_times.shape)
    #Get binned rasters
    raster_dict={}
    time_vec={}
    for n in list(np.unique(spike_clusters)):
        neuron=np.where(spike_clusters==n)
        if len(list(neuron[0]))!=0:
            #print(len(list(neuron[0])))
            raster=trials.get_event_aligned_raster(spike_times[neuron], events, tbin=0.1, values=None, epoch=[-0.5, 2], bin=True)
            #print(raster)
            if np.isnan(raster[0]).any()==False:
                raster_dict[n]=raster[0]
        time_vec=raster[1]
    psth_dict={}
    for k in raster_dict.keys():
        psth=trials.get_psth(raster_dict[k],trial_ids=trials_)
        psth_dict[k]=psth
    return raster_dict,psth_dict,time_vec

raster_dict, psth_dict,time_vec=get_raster_and_psth(eid)

from scipy.stats import zscore
psth_mat=[]
for k in psth_dict.keys():
    psth_mat.append(psth_dict[k][0])
psth_mat=np.array(psth_mat)

np.save('psth_arr_z.npy',psth_mat)

5469038
(5469038,)


In [14]:
import GEMA

import time

start=time.time()
print(start)

from sklearn import preprocessing
map_size = 10
period = 10000 
learning_rate = 0.1
print(np.isnan(psth_mat))
normalize_data = 'none'
datos = preprocessing.normalize(psth_mat)

main_map = GEMA.Map(datos,
                    map_size,
                    period, 
                    initial_lr=learning_rate, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    normalization = normalize_data,
                    presentation= 'random',
                    weights = 'random')

end=time.time()
print(end-start)

#Color SOM
from sklearn import preprocessing
datos_ = np.random.randint(0, 255, (100, 3))
datos_ = preprocessing.normalize(datos_)
main_map_colors = GEMA.Map(datos_, 
                    map_size, 
                    period, 
                    initial_lr=learning_rate, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    normalization = normalize_data,
                    presentation= 'random',
                    weights = 'random')
from matplotlib.pyplot import imshow
image_matrix = (main_map_colors.weights-main_map_colors.weights.min())/(main_map_colors.weights.max()-main_map_colors.weights.min())

plt_arr=[]
for j in range(0,10):
    plt_lst=[]
    for i in range(0,10):
        plt_lst.append(main_map.weights[i,j,:])
    plt_arr.append(plt_lst)
plt_arr=np.array(plt_arr)
print(plt_arr.shape)
np.save('psth_prototypes_arr_z.npy',plt_arr)

import pickle
bmu_dct={}
for i in range(0,10):
    for j in range(0,10):
        bmu_dct[(i,j)]=[]
for k in range(0,datos.shape[0]):
    grid_index=main_map.calculate_bmu(datos[k,:])[1]
    bmu_dct[grid_index].append(k)
with open('bmu_dct_z.pickle', 'wb') as handle:
    pickle.dump(bmu_dct, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(bmu_dct)

alpha_im=np.zeros((10,10,4))
for i in range(0,10):
    for j in range(0,10):
        alpha_im[i,j,:3]=image_matrix[i,j,:]
        if len(bmu_dct[(i,j)])==0:
            alpha_im[i,j,3]=0.5
        else:
            alpha_im[i,j,3]=1
            
np.save('alpha_im_z.npy',alpha_im)

1678624441.9310865
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
TRAINING...


  0%|          | 0/10000 [00:00<?, ?it/s]

FINISHED.
2.334383010864258
TRAINING...


  0%|          | 0/10000 [00:00<?, ?it/s]

FINISHED.
(10, 10, 26)
{(0, 0): [17, 52, 62, 91, 106, 114, 144, 240], (0, 1): [57, 97], (0, 2): [116, 122], (0, 3): [7, 24], (0, 4): [10, 12, 32, 115, 194, 201], (0, 5): [135, 190], (0, 6): [193, 230], (0, 7): [51, 123], (0, 8): [16, 45, 112, 253], (0, 9): [1, 50, 110, 231], (1, 0): [], (1, 1): [19, 71], (1, 2): [], (1, 3): [4, 31, 192, 200, 205, 213], (1, 4): [11, 187], (1, 5): [105, 109], (1, 6): [118], (1, 7): [25, 37, 38, 236], (1, 8): [13, 30], (1, 9): [27, 35, 49, 113, 211], (2, 0): [20, 33, 107, 108, 182, 202], (2, 1): [6, 242], (2, 2): [0, 8, 14, 181], (2, 3): [2, 117], (2, 4): [5, 104, 175, 179], (2, 5): [55, 195], (2, 6): [18, 178], (2, 7): [169], (2, 8): [29, 111, 165, 254], (2, 9): [42], (3, 0): [3], (3, 1): [21, 23, 28, 70, 77, 82], (3, 2): [9, 173], (3, 3): [41, 69, 101, 214, 222], (3, 4): [100, 143, 153, 186, 229], (3, 5): [185], (3, 6): [147], (3, 7): [145], (3, 8): [], (3, 9): [22, 40, 157, 245], (4, 0): [137, 138], (4, 1): [15, 96], (4, 2): [75, 184], (4, 3): [76, 84,

In [13]:
print(image_matrix.shape)

(10, 10, 26)


In [15]:
print(bmu_dct)

{(0, 0): [17, 52, 62, 91, 106, 114, 144, 240], (0, 1): [57, 97], (0, 2): [116, 122], (0, 3): [7, 24], (0, 4): [10, 12, 32, 115, 194, 201], (0, 5): [135, 190], (0, 6): [193, 230], (0, 7): [51, 123], (0, 8): [16, 45, 112, 253], (0, 9): [1, 50, 110, 231], (1, 0): [], (1, 1): [19, 71], (1, 2): [], (1, 3): [4, 31, 192, 200, 205, 213], (1, 4): [11, 187], (1, 5): [105, 109], (1, 6): [118], (1, 7): [25, 37, 38, 236], (1, 8): [13, 30], (1, 9): [27, 35, 49, 113, 211], (2, 0): [20, 33, 107, 108, 182, 202], (2, 1): [6, 242], (2, 2): [0, 8, 14, 181], (2, 3): [2, 117], (2, 4): [5, 104, 175, 179], (2, 5): [55, 195], (2, 6): [18, 178], (2, 7): [169], (2, 8): [29, 111, 165, 254], (2, 9): [42], (3, 0): [3], (3, 1): [21, 23, 28, 70, 77, 82], (3, 2): [9, 173], (3, 3): [41, 69, 101, 214, 222], (3, 4): [100, 143, 153, 186, 229], (3, 5): [185], (3, 6): [147], (3, 7): [145], (3, 8): [], (3, 9): [22, 40, 157, 245], (4, 0): [137, 138], (4, 1): [15, 96], (4, 2): [75, 184], (4, 3): [76, 84, 189, 244], (4, 4): [39

In [17]:
print(alpha_im.shape)

(10, 10, 4)
